This notebook contains an example for teaching.

# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [2]:
data <- read.csv("../data/gun_clean.csv") 
dim(data)[2]

[1] 415

### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [5]:
#################################  Find Variable Names from Dataset ########################

varlist <- function (df=NULL,type=c("numeric","factor","character"), pattern="", exclude=NULL) {
  vars <- character(0)
  if (any(type %in% "numeric")) {
    vars <- c(vars,names(df)[sapply(df,is.numeric)])
  }
  if (any(type %in% "factor")) {
    vars <- c(vars,names(df)[sapply(df,is.factor)])
  }  
  if (any(type %in% "character")) {
    vars <- c(vars,names(df)[sapply(df,is.character)])
  }  
  vars[(!vars %in% exclude) & grepl(vars,pattern=pattern)]
}

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
fixed  <- grep("X_Jfips", names(data), value=TRUE, fixed=TRUE)
year   <- varlist(data, pattern="X_Tyear")

# Census Control Variables
census     <- NULL
census_var <- c("^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS")

for(i in 1:length(census_var)){
  
  census  <- append(census, varlist(data, pattern=census_var[i]))
  
}

################################ Variables ##################################
# Treatment Variable
d     <- "logfssl"

# Outcome Variable
y     <- "logghomr"

# Other Control Variables
X1    <- c("logrobr", "logburg", "burg_missing", "robrate_missing")
X2    <- c("newblack", "newfhh", "newmove", "newdens", "newmal")

#################################  Partial out Fixed Effects ########################

# New Dataset for Partiled-out Variables
rdata    <- as.data.frame(data$CountyCode) 
colnames(rdata) <- "CountyCode"

# Variables to be Partialled-out
varlist <- c(y, d,X1, X2, census)


# Partial out Variables in varlist from year and county fixed effect
for(i in 1:length(varlist)){
  form <- as.formula(paste(varlist[i], "~", paste(paste(year,collapse="+"),  paste(fixed,collapse="+"), sep="+")))
  rdata[, varlist[i]] <- lm(form, data)$residuals
}

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [69]:
# Treatment Variable
D     <- rdata[which(colnames(rdata) == d)]

# Outcome Variable
Y     <- rdata[which(colnames(rdata) == y)]

# Construct matrix Z

Z <- rdata[which(colnames(rdata) %in% c(X1,X2,census))]

We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [72]:
clu <- rdata[which(colnames(rdata) == "CountyCode")] #for clustering the standard errors
data <- data.frame(cbind(Y, D, Z,as.matrix(clu)))

In [74]:
# To use in Python
save(data, file = "../data/gun_clean.RData")

In [76]:
library(lfe)

## The effect of gun ownership

### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [77]:
#baseline_formula <- as.formula(paste(y, "~", d ))
#baseline.ols <- lm(baseline_formula,data=rdata)

baseline.ols <- felm(logghomr ~ logfssl |0|0| CountyCode,data=data) # ols with clustered standard errors
est_baseline <- summary(baseline.ols)$coef[2,]
confint(baseline.ols)[2,]
est_baseline

2.5 %    97.5 % 
0.1552382 0.4093708

Estimate Cluster s.e.      t value     Pr(>|t|) 
2.823045e-01 6.481080e-02 4.355825e+00 1.359767e-05

In [78]:
summary(baseline.ols)


Call:
   felm(formula = logghomr ~ logfssl | 0 | 0 | CountyCode, data = data) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.0162 -0.2253  0.0189  0.2561  1.5753 

Coefficients:
             Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept) 5.189e-18    2.895e-16   0.018    0.986    
logfssl     2.823e-01    6.481e-02   4.356 1.36e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.4199 on 3898 degrees of freedom
Multiple R-squared(full model): 0.006193   Adjusted R-squared: 0.005938 
Multiple R-squared(proj model): 0.006193   Adjusted R-squared: 0.005938 
F-statistic(full model, *iid*):24.29 on 1 and 3898 DF, p-value: 8.623e-07 
F-statistic(proj model): 18.97 on 1 and 194 DF, p-value: 2.146e-05 



In [79]:
confint(baseline.ols)[2,]

2.5 %    97.5 % 
0.1552382 0.4093708

The point estimate is $0.282$ with the confidence interval ranging from 0.155 to 0.41. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [81]:
control_formula <- as.formula(paste("logghomr", "~", paste("logfssl",paste(colnames(Z),collapse="+"),
                                                          sep="+"),"|0|0| CountyCode"))
control_formula

logghomr ~ logfssl + logrobr + logburg + burg_missing + robrate_missing + 
    newblack + newfhh + newmove + newdens + newmal + AGE010D + 
    AGE050D + AGE110D + AGE170D + AGE180D + AGE270D + AGE310D + 
    AGE320D + AGE350D + AGE380D + AGE410D + AGE470D + AGE570D + 
    AGE640D + AGE670D + AGE760D + BNK010D + BNK050D + BPS030D + 
    BPS130D + BPS230D + BPS020D + BPS120D + BPS220D + BPS820D + 
    BZA010D + BZA110D + BZA210D + EDU100D + EDU200D + EDU600D + 
    EDU610D + EDU620D + EDU630D + EDU635D + EDU640D + EDU650D + 
    EDU680D + EDU685D + ELE010D + ELE020D + ELE025D + ELE030D + 
    ELE035D + ELE060D + ELE065D + ELE210D + ELE220D + HIS010D + 
    HIS020D + HIS030D + HIS040D + HIS110D + HIS120D + HIS130D + 
    HIS140D + HIS200D + HIS300D + HIS500D + HIS700D + HSD010D + 
    HSD020D + HSD030D + HSD110D + HSD120D + HSD130D + HSD140D + 
    HSD150D + HSD170D + HSD200D + HSD210D + HSD230D + HSD300D + 
    HSD310D + HSG030D + HSG195D + HSG200D + HSG220D + HSG440D + 
    HSG445D + HS

In [82]:
control.ols <- felm(control_formula,data=data)
est_ols <- summary(control.ols)$coef[2,]
confint(control.ols)[2,]
est_ols

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or indefinite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or indefinite"


2.5 %     97.5 % 
0.08781594 0.29347352

Estimate Cluster s.e.      t value     Pr(>|t|) 
0.1906447266 0.0524475582 3.6349590576 0.0002817867

In [83]:
summary(control.ols)$coef

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or indefinite"


,Estimate,Cluster s.e.,t value,Pr(>|t|)
(Intercept),1.675988e-15,1.659040e-15,1.0102155,0.3124577934
logfssl,1.906447e-01,5.244756e-02,3.6349591,0.0002817867
logrobr,1.890471e-01,5.385086e-02,3.5105680,0.0004524588
logburg,2.192630e-01,6.282510e-02,3.4900546,0.0004885304
burg_missing,1.529614e+00,4.443831e-01,3.4421076,0.0005835660
robrate_missing,1.133081e+00,3.047595e-01,3.7179522,0.0002038034
newblack,2.434718e+01,4.480200e+01,0.5434397,0.5868597027
newfhh,-6.875928e+01,1.146440e+02,-0.5997632,0.5487006184
newmove,4.211082e+01,2.869630e+01,1.4674649,0.1423341971
newdens,NaN,0.000000e+00,NaN,NaN


After controlling for a rich set of characteristics, the point estimate of gun ownership reduces to $0.19$.

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [86]:
I_1 <- c(1,2,3,4,5,6,7,8,9,10)
I_2 <- c(11,12,13,14,15,16,17,18,19,20)
I = c(I_1, I_2)
I

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20

In [87]:
DML2.for.PLM <- function(z, d, y, dreg, yreg, nfold=2, clu) {
  nobs <- nrow(z) #number of observations
  foldid <- rep.int(1:nfold,times = ceiling(nobs/nfold))[sample.int(nobs)] #define folds indices
  I <- split(1:nobs, foldid)  #split observation indices into folds  
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    dfit <- dreg(z[-I[[b]],], d[-I[[b]]]) #take a fold out
    yfit <- yreg(z[-I[[b]],], y[-I[[b]]]) # take a foldt out
    dhat <- predict(dfit, z[I[[b]],], type="response") #predict the left-out fold 
    yhat <- predict(yfit, z[I[[b]],], type="response") #predict the left-out fold  
    dtil[I[[b]]] <- (d[I[[b]]] - dhat) #record residual for the left-out fold
    ytil[I[[b]]] <- (y[I[[b]]] - yhat) #record residial for the left-out fold
    cat(b," ")
        }
  #rfit <- lm(ytil ~ dtil)    #estimate the main parameter by regressing one residual on the other
  data <- data.frame(cbind(ytil, dtil, as.matrix(clu)))
  rfit <- felm(ytil ~ dtil|0|0|CountyCode,data=data) 
  coef.est <- coef(rfit)[2]  #extract coefficient
  #HC <- vcovHC(rfit)
  se    <- summary(rfit,robust=T)$coefficients[2,2] #record robust standard error by County
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est , se))  #printing output
  return( list(coef.est =coef.est , se=se, dtil=dtil, ytil=ytil, rfit=rfit) ) #save output and residuals 
}

Now, we apply the Double Machine Learning (DML) approach with different machine learning methods. First, we load the relevant libraries.

In [88]:
library(hdm)
library(glmnet)
library(sandwich)
library(randomForest)

Let us, construct the input matrices.

In [89]:
y <- as.matrix(Y)
d <- as.matrix(D)
z <- as.matrix(Z)
clu <- rdata[which(colnames(rdata) == "CountyCode")]
head(data.frame(cbind(y,d,as.matrix(clu))))

,logghomr,logfssl,CountyCode
,<dbl>,<dbl>,<dbl>
1,-0.13477752,0.096127077,1073
2,-0.23962152,0.080809373,1073
3,-0.07867716,0.057339916,1073
4,-0.33146546,0.081694483,1073
5,-0.31663980,0.025365514,1073
6,0.10513190,-0.006777264,1073


In the following, we apply the DML approach with the differnt versions of lasso.


## Lasso

In [91]:
#DML with Lasso:
set.seed(123)
dreg <- function(z,d){ rlasso(z,d, post=FALSE) } #ML method= lasso from hdm 
yreg <- function(z,y){ rlasso(z,y, post=FALSE) } #ML method = lasso from hdm

In [92]:
DML2.lasso = DML2.for.PLM(z, d, y, dreg, yreg, nfold=10, clu)

fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = 0.222959 (0.0570325)


In [93]:
#DML with Post-Lasso:
dreg <- function(z,d){ rlasso(z,d, post=T) } #ML method= lasso from hdm 
yreg <- function(z,y){ rlasso(z,y, post=T) } #ML method = lasso from hdm
DML2.post = DML2.for.PLM(z, d, y, dreg, yreg, nfold=10, clu)

fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = 0.226934 (0.0561918)


In [94]:
#DML with cross-validated Lasso:
dreg <- function(z,d){ cv.glmnet(z,d,family="gaussian", alpha=1) } #ML method = lasso from glmnet 
yreg <- function(z,y){ cv.glmnet(z,y,family="gaussian", alpha=1) }  #ML method = lasso from glmnet 
DML2.lasso.cv = DML2.for.PLM(z, d, y, dreg, yreg, nfold=5, clu)

dreg <- function(z,d){ cv.glmnet(z,d,family="gaussian", alpha=0.5) } #ML method = elastic net from glmnet 
yreg <- function(z,y){ cv.glmnet(z,y,family="gaussian", alpha=0.5) }  #ML method = elastic net from glmnet 
DML2.elnet = DML2.for.PLM(z, d, y, dreg, yreg, nfold=5, clu)

dreg <- function(z,d){ cv.glmnet(z,d,family="gaussian", alpha=0) } #ML method = ridge from glmnet 
yreg <- function(z,y){ cv.glmnet(z,y,family="gaussian", alpha=0) }  #ML method = ridge from glmnet 
DML2.ridge = DML2.for.PLM(z, d, y, dreg, yreg, nfold=5, clu)

fold: 1  2  3  4  5  
coef (se) = 0.194926 (0.0569378)
fold: 1  2  3  4  5  
coef (se) = 0.208474 (0.0600804)
fold: 1  2  3  4  5  
coef (se) = 0.200234 (0.0598422)


Here we also compute DML with OLS used as the ML method

In [95]:
dreg <- function(z,d){  glmnet(z,d,family="gaussian", lambda=0) } #ML method = ols from glmnet 
yreg <- function(z,y){  glmnet(z,y,family="gaussian", lambda=0) }  #ML method = ols from glmnet 
DML2.ols = DML2.for.PLM(z, d, y, dreg, yreg, nfold=10, clu)

fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = 0.203079 (0.051136)


Next, we also apply Random Forest for comparison purposes.

### Random Forest


In [66]:
#DML with Random Forest:
dreg <- function(z,d){ randomForest(z, d) } #ML method=Forest 
yreg <- function(z,y){ randomForest(z, y) } #ML method=Forest
set.seed(1)
DML2.RF = DML2.for.PLM(z, d, y, dreg, yreg, nfold=2, clu) # set to 2 due to computation time

fold: 1  2  
coef (se) = 0.153017 (0.0605311)


In [91]:
if (!is.null(d) && !is.factor(d))
             max(floor(ncol(z)/3), 1) else floor(sqrt(ncol(z)))

[1] 65

In [92]:
if (!is.null(d) && !is.factor(d)) 5 else 1

[1] 5

We conclude that the gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by about 0.20% controlling for counties' characteristics.

Finally, let's see which method is actually better. We compute RMSE for predicting D and Y, and see which
of the methods works better.


In [96]:
mods<- list(DML2.ols, DML2.lasso, DML2.post, DML2.lasso.cv, DML2.ridge, DML2.elnet, DML2.RF)

RMSE.mdl<- function(mdl) {
RMSEY <- sqrt(mean(mdl$ytil)^2) 
RMSED <- sqrt(mean(mdl$dtil)^2) 
return( list(RMSEY=RMSEY, RMSED=RMSED))
}

#RMSE.mdl(DML2.lasso)

#DML2.lasso$ytil

Res<- lapply(mods, RMSE.mdl)


prRes.Y<- c( Res[[1]]$RMSEY,Res[[2]]$RMSEY, Res[[3]]$RMSEY, Res[[4]]$RMSEY, Res[[5]]$RMSEY,  Res[[6]]$RMSEY, Res[[7]]$RMSEY)
prRes.D<- c( Res[[1]]$RMSED,Res[[2]]$RMSED, Res[[3]]$RMSED, Res[[4]]$RMSED, Res[[5]]$RMSED, Res[[6]]$RMSED, Res[[7]]$RMSED)

prRes<- rbind(prRes.Y, prRes.D); 
rownames(prRes)<- c("RMSE D", "RMSE Y");
colnames(prRes)<- c("OLS", "Lasso", "Post-Lasso", "CV Lasso", "CV Ridge", "CV Elnet", "RF")
print(prRes,digit=6)

               OLS       Lasso  Post-Lasso    CV Lasso    CV Ridge    CV Elnet
RMSE D 0.000407561 3.25471e-05 1.32656e-04 0.000376929 7.24337e-04 9.66559e-04
RMSE Y 0.000134575 3.35791e-05 6.89649e-05 0.000044933 8.41741e-05 5.80479e-19
               RF
RMSE D 0.01086246
RMSE Y 0.00152755


It looks like the best method for predicting D is Lasso, and the best method for predicting Y is CV Ridge.


In [97]:
dreg <- function(z,d){ rlasso(z,d, post=T) } #ML method= lasso from hdm 
yreg <- function(z,y){ cv.glmnet(z,y,family="gaussian", alpha=0) }  #ML method = ridge from glmnet 
DML2.best= DML2.for.PLM(z, d, y, dreg, yreg, nfold=10, clu)

fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = 0.222066 (0.0565614)


Let's organize the results in a table.

In [84]:
est_baseline

Estimate Cluster s.e.      t value     Pr(>|t|) 
2.823045e-01 6.481080e-02 4.355825e+00 1.359767e-05

In [18]:
library(xtable)

table <- matrix(0,9,2)
table[1,1] <- as.numeric(est_baseline[1])
table[2,1] <- as.numeric(est_ols[1])
table[3,1]   <- as.numeric(DML2.lasso$coef.est)
table[4,1]   <- as.numeric(DML2.post$coef.est)
table[5,1]  <-as.numeric(DML2.lasso.cv$coef.est)
table[6,1] <-as.numeric(DML2.elnet$coef.est)
table[7,1] <-as.numeric(DML2.ridge$coef.est)
table[8,1] <-as.numeric(DML2.RF$coef.est)
table[9,1] <-as.numeric(DML2.best$coef.est)
table[1,2] <- as.numeric(est_baseline[2])
table[2,2] <- as.numeric(est_ols[2])
table[3,2]   <- as.numeric(DML2.lasso$se)
table[4,2]   <- as.numeric(DML2.post$se)
table[5,2]  <-as.numeric(DML2.lasso.cv$se)
table[6,2] <-as.numeric(DML2.elnet$se)
table[7,2] <-as.numeric(DML2.ridge$se)
table[8,2] <-as.numeric(DML2.RF$se)
table[9,2] <-as.numeric(DML2.best$se)




################################# Print Results #################################

colnames(table) <- c("Estimate","Standard Error")
rownames(table) <- c("Baseline OLS", "Least Squares with controls", "Lasso", "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Random Forest", 
                     "Best")

table

,Estimate,Standard Error
Baseline OLS,0.2823045,0.06481080
Least Squares with controls,0.1906447,0.05244756
Lasso,0.2228074,0.05702673
Post-Lasso,0.2269338,0.05619181
CV Lasso,0.2004742,0.05764115
CV Elnet,0.2061170,0.05746222
CV Ridge,0.2013789,0.05790663
Random Forest,0.1921739,0.05814101
Best,0.2190048,0.05721956


In [19]:
print(table, digit=3)


                            Estimate Standard Error
Baseline OLS                   0.282         0.0648
Least Squares with controls    0.191         0.0524
Lasso                          0.223         0.0570
Post-Lasso                     0.227         0.0562
CV Lasso                       0.200         0.0576
CV Elnet                       0.206         0.0575
CV Ridge                       0.201         0.0579
Random Forest                  0.192         0.0581
Best                           0.219         0.0572


In [20]:
tab<- xtable(table, digits=3)
print(tab, type="latex")

% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Feb 13 17:41:19 2021
\begin{table}[ht]
\centering
\begin{tabular}{rrr}
  \hline
 & Estimate & Standard Error \\ 
  \hline
Baseline OLS & 0.282 & 0.065 \\ 
  Least Squares with controls & 0.191 & 0.052 \\ 
  Lasso & 0.223 & 0.057 \\ 
  Post-Lasso & 0.227 & 0.056 \\ 
  CV Lasso & 0.200 & 0.058 \\ 
  CV Elnet & 0.206 & 0.057 \\ 
  CV Ridge & 0.201 & 0.058 \\ 
  Random Forest & 0.192 & 0.058 \\ 
  Best & 0.219 & 0.057 \\ 
   \hline
\end{tabular}
\end{table}
